# database class

In [1]:
import sqlite3
import datetime
import random
import string

class Database:
    # implement database as a singleton
    # connection can be found in Database.instance.connection
    class __Database:
        def __init__(self, dbName):
            self.databaseName = dbName

            self.connection = sqlite3.connect(self.databaseName)
            cursor = self.connection.cursor()

            cursor.execute('CREATE TABLE IF NOT EXISTS Users(UserID INTEGER PRIMARY KEY, Name TEXT)')
            cursor.execute('CREATE TABLE IF NOT EXISTS Items(ItemID INTEGER PRIMARY KEY, Name TEXT)')
            cursor.execute('CREATE TABLE IF NOT EXISTS Recommenders(RecommenderID INTEGER PRIMARY KEY, Name TEXT)')
            #cursor.execute("""
            #  CREATE TABLE IF NOT EXISTS RecommendedItems(
            #  RecommendationID integer PRIMARY KEY,
            #  UserID INTEGER REFERENCES Users(UserID),
            #  ItemID INTEGER REFERENCES Items(ItemID),
            #  RecommenderID INTEGER REFERENCES Recommenders(RecommenderID),
            #  Timestamp TIMESTAMP,
            #  Position INTEGER,
            #  Clicked BOOLEAN)
            #  """)

            #simplified for the off-line evaluations
            cursor.execute("""
              CREATE TABLE IF NOT EXISTS RecommendedItems(
              RecommendationID integer PRIMARY KEY,
              UserID INTEGER,
              ItemID INTEGER,
              RecommenderID INTEGER,
              Timestamp TIMESTAMP,
              Position INTEGER,
              Clicked BOOLEAN)
              """)
            cursor.execute("""
                CREATE INDEX IF NOT EXISTS uid 
                ON RecommendedItems(UserID);
            """)
            cursor.execute("""
                CREATE INDEX IF NOT EXISTS tmstmp 
                ON RecommendedItems(Timestamp);
            """)

            self.connection.commit()



        def insertUser(self, username):
            
            #connection = sqlite3.connect(self.databaseName)
            cursor = self.connection.cursor()

            cursor.execute("INSERT INTO Users (Name) VALUES (?)", (username,))

            self.connection.commit()
            #connection.close()
            

        def insertItem(self, name):
            
            #connection = sqlite3.connect(self.databaseName)
            cursor = self.connection.cursor()

            cursor.execute("INSERT INTO Items (Name) VALUES (?)", (name,))

            self.connection.commit()
            #connection.close()
            

        def insertRecommendation(self, userID, itemID, RecommenderID, position, clicked, timestamp = datetime.datetime.now()):
            #connection = sqlite3.connect(self.databaseName)
            cursor = self.connection.cursor()

            cursor.execute("INSERT INTO RecommendedItems (UserID, ItemID, RecommenderID, Timestamp, Position, Clicked) VALUES (?,?,?,?,?,?)", (userID, itemID, RecommenderID, timestamp, position, clicked))

            self.connection.commit()
            #connection.close()

          # currently only recommeder 1
        def getPreviousRecommendations(self, userID, limit = 100):
            #connection = sqlite3.connect(self.databaseName)
            cursor = self.connection.cursor()

            cursor.execute("SELECT * FROM RecommendedItems WHERE UserID=? ORDER BY Timestamp desc LIMIT ?",(userID,limit))
            previousRecommendations = cursor.fetchall()

            self.connection.commit()
            #connection.close()

            return previousRecommendations

        def getInteractionCount(self, userID, limit):
            #connection = sqlite3.connect(self.databaseName)
            cursor = self.connection.cursor()

            cursor.execute("SELECT * FROM RecommendedItems WHERE  UserID=? ORDER BY Timestamp desc LIMIT ?",(userID,limit))
            previousRecommendations = cursor.fetchall()

            self.connection.commit()
            #connection.close()

            return len(previousRecommendations)



        def generateRandomData(self, userCount, itemCount, recommenderCount, sessionsPerWeek, recommendationsPerSession, recommenderOverlap, itemsPerRecommendation):
            #connection = sqlite3.connect(self.databaseName)
            cursor = self.connection.cursor()

            # generate users
            for i in range(userCount):
              username = "U" + "".join(random.choice(string.ascii_lowercase) for j in range(8))
              cursor.execute("INSERT INTO Users (Name) VALUES (?)", (username,))

            # generate items
            for i in range(itemCount):
              name = "I" + "".join(random.choice(string.ascii_lowercase) for j in range(8))
              cursor.execute("INSERT INTO Items (Name) VALUES (?)", (name,))

            # generate recommenders
            for i in range(recommenderCount):
              cursor.execute("INSERT INTO Recommenders (Name) VALUES (?)", (i,))


            cursor.execute("SELECT * FROM Users")
            users = cursor.fetchall()

            cursor.execute("SELECT * FROM Items")
            items = cursor.fetchall()

            cursor.execute("SELECT * FROM Recommenders")
            recommenders = cursor.fetchall()

            for j in range(sessionsPerWeek):
              user = random.choice(users)
              for k in range(recommendationsPerSession):
                time = randomDatetime(datetime.datetime(2020, 3, 23, 0, 0, 0), datetime.datetime(2020, 3, 30, 8, 59, 59))
                for l in range(recommenderOverlap):
                  item = random.choice(items)
                  position = random.choice(range(itemsPerRecommendation))
                  for recommender in recommenders:
                    cursor.execute(
                      "INSERT INTO RecommendedItems (UserID, ItemID, RecommenderID, Timestamp, Position, Clicked) VALUES (?,?,?,?,?,?)",
                      (user[0], item[0], recommender[0], time + l * datetime.timedelta(seconds=120), position, False))
                for l in range(itemsPerRecommendation - recommenderOverlap):
                  item = random.choice(items)
                  position = random.choice(range(itemsPerRecommendation))
                  recommender = random.choice(recommenders)
                  cursor.execute(
                    "INSERT INTO RecommendedItems (UserID, ItemID, RecommenderID, Timestamp, Position, Clicked) VALUES (?,?,?,?,?,?)",
                    (user[0], item[0], recommender[0], time + l * datetime.timedelta(seconds=120), position, False))

            self.connection.commit()
            #connection.close()            
            
            
            
    instance = None
    
    def __init__(self, dbName):
        if not Database.instance:
            Database.instance = Database.__Database(dbName)
            
            
def randomDatetime(start, end):
  '''
  Generates a random time (precise to seconds) between start and end
  :param start: datetime.datetime object
  :param end: datetime.datetime object
  :return: datetime.datetime object with time between start and end
  '''

  if type(start) is not datetime.datetime:
    raise ValueError("Argument start is not type datetime.datetime")
  if type(end) is not datetime.datetime:
    raise ValueError("Argument end is not type datetime.datetime")


  timeFrame = end - start
  seconds = (timeFrame.days * 24 * 60 * 60) + timeFrame.seconds
  randomSecond = random.randrange(seconds)
  return start + datetime.timedelta(seconds=randomSecond)


# Elections class

In [40]:
import sys
sys.path.insert(0, '..')

import random

import numpy as np
import pandas as pd

from numpy.random import beta
from configuration.arguments import Arguments #class
from configuration.argument import Argument #class

from recommendation.resultOfRecommendation import ResultOfRecommendation #class
from recommendation.resultsOfRecommendations import ResultsOfRecommendations #class


class AggrElections:

    # arguments:Arguments
    def __init__(self, arguments):

       #(if type(arguments) is not Arguments:
       #   raise ValueError("Argument arguments is not type Arguments.")

       #self._arguments = arguments;
       self.db = Database("test3.db")

    def penalizeIfRecentlyRecommended(self, previousRecommendations, itemID, 
                                      maxFinalPenalty = 0.95,
                                      minTimeDiff = 10, #2*60*60, # 2 hours
                                      maxTimeDiff = 100, #7*24*60*60, # 7 days
                                      minSinglePenalty= 0.50,
                                      maxSinglePenalty=1.00,
                                      usePositions = True,
                                      maxPositionPenalty = 1.0):
      '''
      reduces given score based on how many times has the item already been recommended to the user in the last week.
      :param previousRecommendations: array of previous recommendations made to our user. Output of getPreviousRecommendations
      :param itemID: item id
      :param maxFinalPenalty: maximal penalty given to an item
      :param minTimeDiff: single recommendation penalty starts to decrease after this age (seconds)
      :param maxTimeDiff: single recommendation penalty remains minimal after this age (seconds)
      :param minSinglePenalty: minimal single recommendation penalty
      :param maxSinglePenalty: maximal single recommendation penalty
      :return: penalized score
      '''
      if (minTimeDiff < 0):
        raise ValueError("minTimeDiff must not be negative")
      if (maxTimeDiff <= minTimeDiff):
        raise ValueError("maxTimeDiff must be greater than minTimeDiff")
      if (minSinglePenalty < 0):
        raise ValueError("minPenalty must not be negative")
      if (maxSinglePenalty < minSinglePenalty):
        raise ValueError("maxPenalty must be greater than or equal to minPenalty")


      penalty = 0
      #temporal penalty discount-> use recommendation[6] as a timeDiff
      #linear penalty discount -> use counter
      #use recommended position (recommendation[7]) in cases, where you want penalty discount based on positions
      counter = 0
      last_timestamp = 0
      for recommendation in previousRecommendations:
        if recommendation[4] != last_timestamp:
            last_timestamp = recommendation[4]
            counter += 1
        if (recommendation[2] == itemID):
          timeDiff = counter
          if usePositions:
                position = recommendation[5]
                penalty += self.getPenaltyPosition(position, maxPositionPenalty) * self.getPenaltyLinear(timeDiff, minTimeDiff, maxTimeDiff, minSinglePenalty, maxSinglePenalty)
          else:
              penalty += self.getPenaltyLinear(timeDiff, minTimeDiff, maxTimeDiff, minSinglePenalty, maxSinglePenalty)

      return penalty


    def getPenaltyPosition(self, position, maxPenalty):
        """
        Simpler version of penalty for 1/1+p position discount
        """
               
        return maxPenalty / (1+position)
        

    def getPenaltyLinear(self, timeDiff, minTimeDiff, maxTimeDiff, minPenalty, maxPenalty):
      '''
      computes linear penalty based on a line equation
      y = m*x + c   ~   penalty = m*timeDiff + c
      the line is defined by two points:
      (minTimeDiff, maxPenalty), (maxTimeDiff, minPenalty)
      :param timeDiff: time since the recommendation (in seconds)
      :param minTimeDiff: penalty starts to decrease after this time
      :param maxTimeDiff: penalty remains minimal after this time
      :param minPenalty: minimal penalty given (for timeDiff >= maxTimeDiff)
      :param maxPenalty: maximal penalty given (for timeDiff <= minTimeDiff)
      :return: computed penalty
      '''
      if (timeDiff <= 0):
        raise ValueError("timeDiff must not be negative")
      if (minTimeDiff < 0):
        raise ValueError("minTimeDiff must not be negative")
      if (maxTimeDiff <= minTimeDiff):
        raise ValueError("maxTimeDiff must be greater than minTimeDiff")
      if (minPenalty < 0):
        raise ValueError("minPenalty must not be negative")
      if (maxPenalty < minPenalty):
        raise ValueError("maxPenalty must be greater than or equal to minPenalty")

      m = (minPenalty - maxPenalty) / (maxTimeDiff - minTimeDiff)
      c = maxPenalty - (m * maxTimeDiff)
      res = (m * timeDiff) + c
      if res < minPenalty:
            res = minPenalty
      elif res > maxPenalty:
            res = maxPenalty
    
      return res
    
    
    # resultsOfRecommendations:ResultsOfRecommendations, evaluationOfRecommenders:EvaluationOfRecommenders, numberOfItems:int
    def run(self, resultsOfRecommendations, evaluationOfRecommenders, numberOfItems=20):

        #if type(resultsOfRecommendations) is not ResultsOfRecommendations:
        #     raise ValueError("Argument resultsOfRecommendations is not type ResultsOfRecommendations.")

        if type(numberOfItems) is not int:
             raise ValueError("Argument numberOfItems is not type int.")

        methodsResultDict = resultsOfRecommendations.exportAsDictionaryOfSeries()
        #print(methodsResultDict)

        methodsParamsDF = evaluationOfRecommenders.exportAsParamsDF()
        #print(methodsParamsDF)

        return self.aggrElectionsRun(methodsResultDict, methodsParamsDF, topK=numberOfItems)


    # methodsResultDict:{String:pd.Series(rating:float[], itemID:int[])},
    # methodsParamsDF:pd.DataFrame[numberOfVotes:int], topK:int
    def aggrElectionsRun(self, methodsResultDict, methodsParamsDF, userID, topK = 20, penalize=True):

      if sorted([mI for mI in methodsParamsDF.index]) != sorted([mI for mI in methodsResultDict.keys()]):
        raise ValueError("Arguments methodsResultDict and methodsParamsDF have to define the same methods.")

      if np.prod([len(methodsResultDict.get(mI)) for mI in methodsResultDict]) == 0:
        raise ValueError("Argument methodsParamsDF contains in ome method an empty list of items.")

      if topK < 0 :
        raise ValueError("Argument topK must be positive value.")

      candidatesOfMethods = np.asarray([cI.keys() for cI in methodsResultDict.values()])
      uniqueCandidatesI = list(set(np.concatenate(candidatesOfMethods)))
      #print("UniqueCandidatesI: ", uniqueCandidatesI)

      # numbers of elected candidates of parties
      electedOfPartyDictI = {mI:1 for mI in methodsParamsDF.index}
      #print("ElectedForPartyI: ", electedOfPartyDictI)

      # votes number of parties
      votesOfPartiesDictI = {mI:methodsParamsDF.votes.loc[mI] for mI in methodsParamsDF.index}
      #print("VotesOfPartiesDictI: ", votesOfPartiesDictI)

      recommendedItemIDs = []
        
      #calculate candidates penalties
      penalties = {}
      if penalize:
        previousRecommendations = self.db.instance.getPreviousRecommendations(userID, limit = 500)
        for itemID in uniqueCandidatesI:                    
            penalties[itemID] = self.penalizeIfRecentlyRecommended(previousRecommendations, itemID)
        #print("Penalties:",penalties)
        
      for iIndex in range(0, topK):
        #print("iIndex: ", iIndex)

        if len(uniqueCandidatesI) == 0:
            return recommendedItemIDs[:topK]

        # coumputing of votes of remaining candidates
        actVotesOfCandidatesDictI = {}
        for candidateIDJ in uniqueCandidatesI:
           votesOfCandidateJ = 0
           for parityIDK in methodsParamsDF.index:
              partyAffiliationOfCandidateKJ = methodsResultDict[parityIDK].get(candidateIDJ, 0)
              votesOfPartyK = votesOfPartiesDictI.get(parityIDK)
              votesOfCandidateJ += partyAffiliationOfCandidateKJ * votesOfPartyK           
           if penalize:
                actVotesOfCandidatesDictI[candidateIDJ] = votesOfCandidateJ / (1 + penalties.get(candidateIDJ, 0))
           else:
                actVotesOfCandidatesDictI[candidateIDJ] = votesOfCandidateJ
                
        #print("CandidateVotes: ",actVotesOfCandidatesDictI)
        
        #print(list(actVotesOfCandidatesDictI.values()))
        
        # get the highest number of votes of remaining candidates
        maxVotes = np.max(list(actVotesOfCandidatesDictI.values()))
        #print("MaxVotes: ", maxVotes)

        # select candidate with highest number of votes
        selectedCandidateI = [votOfCandI for votOfCandI in actVotesOfCandidatesDictI.keys() if actVotesOfCandidatesDictI[votOfCandI] == maxVotes][0]
        #print("SelectedCandidateI: ", selectedCandidateI)

        # add new selected candidate in results
        recommendedItemIDs.append(selectedCandidateI);

        # removing elected candidate from list of candidates
        uniqueCandidatesI.remove(selectedCandidateI)

        # updating number of elected candidates of parties
        electedOfPartyDictI = {partyIDI:electedOfPartyDictI[partyIDI] + methodsResultDict[partyIDI].get(selectedCandidateI, 0) for partyIDI in electedOfPartyDictI.keys()}
        #print("DevotionOfPartyDictI: ", electedOfPartyDictI)

        # updating number of votes of parties
        votesOfPartiesDictI = {partyI:methodsParamsDF.votes.loc[partyI] / electedOfPartyDictI.get(partyI)  for partyI in methodsParamsDF.index}
        #print("VotesOfPartiesDictI: ", votesOfPartiesDictI)

      return recommendedItemIDs[:topK]

    # methodsResultDict:{String:pd.Series(rating:float[], itemID:int[])},
    # methodsParamsDF:pd.DataFrame[numberOfVotes:int], topK:int
    # differencAmplificatorExponent : int / float
    def aggrRandomizedElectionsRun(self, methodsResultDict, methodsParamsDF, differenceAmplificatorExponent, topK=20):

        if sorted([mI for mI in methodsParamsDF.index]) != sorted([mI for mI in methodsResultDict.keys()]):
            raise ValueError("Arguments methodsResultDict and methodsParamsDF have to define the same methods.")

        if np.prod([len(methodsResultDict.get(mI)) for mI in methodsResultDict]) == 0:
            raise ValueError("Argument methodsParamsDF contains in ome method an empty list of items.")

        if topK < 0:
            raise ValueError("Argument topK must be positive value.")

        candidatesOfMethods = np.asarray([cI.keys() for cI in methodsResultDict.values()])
        uniqueCandidatesI = list(set(np.concatenate(candidatesOfMethods)))
        # print("UniqueCandidatesI: ", uniqueCandidatesI)

        # numbers of elected candidates of parties
        electedOfPartyDictI = {mI: 1 for mI in methodsParamsDF.index}
        # print("ElectedForPartyI: ", electedOfPartyDictI)

        # votes number of parties
        votesOfPartiesDictI = {mI: methodsParamsDF.votes.loc[mI] for mI in methodsParamsDF.index}
        # print("VotesOfPartiesDictI: ", votesOfPartiesDictI)

        recommendedItemIDs = []

        for iIndex in range(0, topK):
            # print("iIndex: ", iIndex)

            if len(uniqueCandidatesI) == 0:
                return recommendedItemIDs[:topK]

            # coumputing of votes of remaining candidates
            actVotesOfCandidatesDictI = {}
            for candidateIDJ in uniqueCandidatesI:
                votesOfCandidateJ = 0
                for parityIDK in methodsParamsDF.index:
                    partyAffiliationOfCandidateKJ = methodsResultDict[parityIDK].get(candidateIDJ, 0)
                    votesOfPartyK = votesOfPartiesDictI.get(parityIDK)
                    votesOfCandidateJ += partyAffiliationOfCandidateKJ * votesOfPartyK
                actVotesOfCandidatesDictI[candidateIDJ] = votesOfCandidateJ ** differenceAmplificatorExponent
            # print(actVotesOfCandidatesDictI)

            # ROULETTE SELECTION (selecting random candidate according vote distribution)
            # compute sum of all votes
            voteSum = sum(actVotesOfCandidatesDictI.values())

            selection_probs = [c/voteSum for c in actVotesOfCandidatesDictI.values()]
            selected_index = np.random.choice(len(actVotesOfCandidatesDictI), p=selection_probs)
            selectedCandidateI = list(actVotesOfCandidatesDictI.keys())[selected_index]
            
            # random number in range <0, voteSum)
            # rnd = random.randrange(0, (int)(voteSum), 1)
            # find random candidate
            #for candidate in actVotesOfCandidatesDictI.keys():
            #    if (rnd < actVotesOfCandidatesDictI[candidate]):
            #        selectedCandidateI = candidate
            #        break
            #    rnd -= actVotesOfCandidatesDictI[candidate]

            # add new selected candidate in results
            recommendedItemIDs.append(selectedCandidateI);

            # removing elected candidate from list of candidates
            uniqueCandidatesI.remove(selectedCandidateI)

            # updating number of elected candidates of parties
            electedOfPartyDictI = {
                partyIDI: electedOfPartyDictI[partyIDI] + [partyIDI].get(selectedCandidateI, 0) for
                partyIDI in electedOfPartyDictI.keys()}
            # print("DevotionOfPartyDictI: ", devotionOfPartyDictI)

            # updating number of votes of parties
            votesOfPartiesDictI = {partyI: methodsParamsDF.votes.loc[partyI] / electedOfPartyDictI.get(partyI) for
                partyI in methodsParamsDF.index}
            # print("VotesOfPartiesDictI: ", votesOfPartiesDictI)

        return recommendedItemIDs[:topK]


    # methodsResultDict:{String:pd.Series(rating:float[], itemID:int[])},
    # methodsParamsDF:pd.DataFrame[numberOfVotes:int], topK:int
    def aggrElectionsRunWithResponsibility(self, methodsResultDict, methodsParamsDF, topK = 20):
      
      # recommendedItemIDs:int[]
      recommendedItemIDs = self.aggrElectionsRun(methodsResultDict, methodsParamsDF, topK)
      votesOfPartiesDictI = {mI:methodsParamsDF.votes.loc[mI] for mI in methodsParamsDF.index}
      
      candidatesOfMethods = np.asarray([cI.keys() for cI in methodsResultDict.values()])
      uniqueCandidatesI = list(set(np.concatenate(candidatesOfMethods)))

      candidateOfdevotionOfPartiesDictDict = {}
      for candidateIDI in recommendedItemIDs:
      #for candidateIDI in uniqueCandidatesI:
         devotionOfParitiesDict = {}
         for parityIDJ in methodsParamsDF.index:
            devotionOfParitiesDict[parityIDJ] = methodsResultDict[parityIDJ].get(candidateIDI, 0)  * votesOfPartiesDictI[parityIDJ]
         candidateOfdevotionOfPartiesDictDict[candidateIDI] = devotionOfParitiesDict
      #print(candidateOfdevotionOfPartiesDictDict)

      # selectedCandidate:[itemID:{methodID:responsibility,...},...]
      selectedCandidate = [(candidateI, candidateOfdevotionOfPartiesDictDict[candidateI]) for candidateI in recommendedItemIDs]

      return selectedCandidate


# testování agregovaných modelů

In [3]:
db = Database("test4.db")

In [4]:
db.instance.generateRandomData(500, 500, 5, 2500, 10, 20, 50)


In [5]:
userID = 2
limit = 5000

In [6]:
db.instance.getInteractionCount(userID, limit)

3900

In [7]:
import timeit
timeit.timeit('db.instance.getPreviousRecommendations(userID, limit)', number=100, globals=globals())

1.411336899999995

## vytvorim si nahodny dotaz slozeny ze 3 doporucovacich metod

In [31]:
  N = 20
  from sklearn import preprocessing
  # method results, items=[1,2,4,5,6,7,8,12,32,64,77]
  methodsResultDict = {
          "metoda1":pd.Series(preprocessing.normalize(np.random.uniform(size=10).reshape(1, -1), norm='l2').flatten(),np.random.choice(500,size=10, replace=False),name="rating"),
          "metoda2":pd.Series(preprocessing.normalize(np.random.uniform(size=10).reshape(1, -1), norm='l2').flatten(),np.random.choice(500,size=10, replace=False),name="rating"),
          "metoda3":pd.Series(preprocessing.normalize(np.random.uniform(size=10).reshape(1, -1), norm='l2').flatten(),np.random.choice(500,size=10, replace=False),name="rating"),
          }
  #print(methodsResultDict)


  # methods parametes
  methodsParamsData = [['metoda1',1.0], ['metoda2',0.8], ['metoda3',0.6]]
  methodsParamsDF = pd.DataFrame(methodsParamsData, columns=["methodID","votes"])
  methodsParamsDF.set_index("methodID", inplace=True)

  methodsParamsDF


  

,votes
methodID,
metoda1,1.0
metoda2,0.8
metoda3,0.6


In [41]:

  args = Arguments([Argument("arg1", 0)])
  aggregator = AggrElections(args)

In [38]:
methodsResultDict

{'metoda1': 200    0.156757
 444    0.368296
 497    0.381064
 174    0.321240
 298    0.156971
 53     0.412238
 292    0.232298
 108    0.023443
 216    0.458383
 374    0.363557
 Name: rating, dtype: float64,
 'metoda2': 436    0.068143
 216    0.243783
 109    0.319127
 27     0.287170
 459    0.080453
 304    0.546283
 158    0.341702
 144    0.216311
 242    0.530322
 351    0.043912
 Name: rating, dtype: float64,
 'metoda3': 416    0.655876
 68     0.261204
 299    0.240582
 357    0.449472
 18     0.317734
 346    0.097624
 32     0.129864
 176    0.088742
 326    0.038768
 48     0.323987
 Name: rating, dtype: float64}

### vysledek pro hlasovani s vyuzitim penalizace drive zobrazenych objektu

In [42]:
aggregator.aggrElectionsRun(methodsResultDict, methodsParamsDF,2, N, True)

[216,
 416,
 242,
 53,
 304,
 497,
 444,
 357,
 374,
 158,
 109,
 48,
 174,
 18,
 292,
 27,
 68,
 144,
 200,
 299]

In [47]:
### vysledek pro hlasovani bez vyuziti penalizace drive zobrazenych objektu

In [43]:
aggregator.aggrElectionsRun(methodsResultDict, methodsParamsDF,2, N, False)

[216,
 416,
 304,
 53,
 242,
 497,
 444,
 357,
 374,
 158,
 174,
 109,
 48,
 18,
 27,
 292,
 68,
 144,
 299,
 298]

In [45]:
timeit.timeit('aggregator.aggrElectionsRun(methodsResultDict, methodsParamsDF,2, N, True)', number=100, globals=globals())

2.910839699999997

In [46]:
timeit.timeit('aggregator.aggrElectionsRun(methodsResultDict, methodsParamsDF,2, N, False)', number=100, globals=globals())

2.068192399999816

In [112]:
aggregator.aggrElectionsRunWithResponsibility(methodsResultDict, methodsParamsDF, N)

UniqueCandidatesI:  [387, 134, 7, 393, 394, 13, 284, 169, 427, 436, 53, 440, 445, 446, 202, 458, 208, 340, 89, 475, 355, 101, 365, 367, 368, 244, 246, 248, 254]
ElectedForPartyI:  {'metoda1': 1, 'metoda2': 1, 'metoda3': 1}
VotesOfPartiesDictI:  {'metoda1': 1.0, 'metoda2': 0.8, 'metoda3': 0.6}
CandidateVotes:  {387: 0.141872995903562, 134: 0.16262062320512716, 7: 0.3470697128925702, 393: 0.20063464674653636, 394: 0.46467653680700505, 13: 0.17991608504103862, 284: 0.02983361537541076, 169: 0.14541822574061664, 427: 0.005297306088443672, 436: 0.3611478093988654, 53: 0.195847882951716, 440: 0.05584046886695443, 445: 0.24981392703489733, 446: 0.2737195052541701, 202: 0.11159668130526708, 458: 0.0938648367464995, 208: 0.32301358150516807, 340: 0.2563847789424132, 89: 0.2827272458359441, 475: 0.24591854092260523, 355: 0.31628957862038326, 101: 0.4640144018995287, 365: 0.5659502746957056, 367: 0.14757591582776264, 368: 0.2795815175044896, 244: 0.1016332386763398, 246: 0.21692587599661564, 248:

[(365, {'metoda1': 0.5659502746957056, 'metoda2': 0.0, 'metoda3': 0.0}),
 (436, {'metoda1': 0.0, 'metoda2': 0.3611478093988654, 'metoda3': 0.0}),
 (355, {'metoda1': 0.0, 'metoda2': 0.0, 'metoda3': 0.31628957862038326}),
 (394, {'metoda1': 0.46467653680700505, 'metoda2': 0.0, 'metoda3': 0.0}),
 (254, {'metoda1': 0.0, 'metoda2': 0.3516151301390657, 'metoda3': 0.0}),
 (101, {'metoda1': 0.4640144018995287, 'metoda2': 0.0, 'metoda3': 0.0}),
 (7, {'metoda1': 0.0, 'metoda2': 0.3470697128925702, 'metoda3': 0.0}),
 (446, {'metoda1': 0.0, 'metoda2': 0.0, 'metoda3': 0.2737195052541701}),
 (208, {'metoda1': 0.0, 'metoda2': 0.32301358150516807, 'metoda3': 0.0}),
 (340, {'metoda1': 0.0, 'metoda2': 0.0, 'metoda3': 0.2563847789424132}),
 (368,
  {'metoda1': 0.16583971323163763,
   'metoda2': 0.0,
   'metoda3': 0.11374180427285199}),
 (89, {'metoda1': 0.2827272458359441, 'metoda2': 0.0, 'metoda3': 0.0}),
 (445, {'metoda1': 0.0, 'metoda2': 0.0, 'metoda3': 0.24981392703489733}),
 (475, {'metoda1': 0.0, '